# Tutorial 03: ML Categories and Task Selection

Welcome to the fourth tutorial in our ML System Design series! This tutorial provides a deep dive into different ML categories and helps you choose the right approach for your problem.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. **Understand** supervised, unsupervised, and reinforcement learning paradigms
2. **Distinguish** between classification, regression, and ranking tasks
3. **Make informed decisions** about ML category selection
4. **Implement** basic examples of each category

---

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from dataclasses import dataclass
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
np.random.seed(42)

print("Setup complete!")

## ML Categories Overview

Machine Learning can be broadly categorized into three main paradigms.

In [ ]:
# Summary table
ml_categories = pd.DataFrame({
    'Category': ['Supervised', 'Unsupervised', 'Reinforcement'],
    'Data Requirement': ['Labeled data (X, y)', 'Unlabeled data (X only)', 'Environment + Rewards'],
    'Learning Signal': ['Ground truth labels', 'Data structure/patterns', 'Reward signal'],
    'Goal': ['Predict labels for new data', 'Discover hidden patterns', 'Maximize cumulative reward'],
    'Common Tasks': ['Classification, Regression', 'Clustering, Dim Reduction', 'Game playing, Robotics'],
    'Example': ['Spam detection', 'Customer segmentation', 'Recommendation optimization']
})

print("ML Categories Comparison:")
display(ml_categories)

---

## 1. Supervised Learning

Supervised learning is the most common paradigm where we learn from labeled examples.

### 1.1 Binary Classification

In [ ]:
# Example: Spam Detection (Binary Classification)
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

emails = [
    "Congratulations! You've won a free iPhone!",
    "Meeting reminder: Team sync at 3pm today",
    "URGENT: Your account has been compromised!",
    "Hi John, can you review the Q3 report?",
    "FREE MONEY! No credit check required!",
    "Project update: We're on track for Friday",
]

labels = [1, 0, 1, 0, 1, 0]  # 1 = spam, 0 = not spam

vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(emails)
y = np.array(labels)

model = LogisticRegression()
model.fit(X, y)

test_emails = ["Win a free vacation!", "Please review the document"]
X_test = vectorizer.transform(test_emails)
predictions = model.predict(X_test)

print("Binary Classification: Spam Detection")
print("=" * 50)
for email, pred in zip(test_emails, predictions):
    print(f"Email: '{email}'")
    print(f"Prediction: {'SPAM' if pred == 1 else 'NOT SPAM'}\n")

### 1.2 Multi-class Classification

In [ ]:
# Example: Sentiment Analysis (Multi-class)
from sklearn.naive_bayes import MultinomialNB

reviews = [
    "This product is amazing! Best purchase ever!",
    "Terrible quality. Waste of money.",
    "It's okay, nothing special.",
    "Absolutely love it! Exceeded expectations!",
    "Disappointed. Doesn't work as advertised.",
    "Decent product for the price."
]

sentiments = [2, 0, 1, 2, 0, 1]  # 0=negative, 1=neutral, 2=positive
sentiment_labels = ['Negative', 'Neutral', 'Positive']

vectorizer = TfidfVectorizer(max_features=100)
X = vectorizer.fit_transform(reviews)
model = MultinomialNB()
model.fit(X, sentiments)

test_reviews = ["Really enjoyed this!", "Not great, not terrible."]
X_test = vectorizer.transform(test_reviews)
predictions = model.predict(X_test)

print("Multi-class Classification: Sentiment Analysis")
print("=" * 50)
for review, pred in zip(test_reviews, predictions):
    print(f"Review: '{review}'")
    print(f"Sentiment: {sentiment_labels[pred]}\n")

### 1.3 Regression

In [ ]:
# Example: House Price Prediction (Regression)
from sklearn.ensemble import GradientBoostingRegressor

np.random.seed(42)
n_samples = 200

sqft = np.random.randint(800, 4000, n_samples)
bedrooms = np.random.randint(1, 6, n_samples)
age = np.random.randint(0, 50, n_samples)

price = sqft * 200 + bedrooms * 20000 - age * 1000 + np.random.normal(0, 30000, n_samples)
price = np.maximum(price, 50000)

X = np.column_stack([sqft, bedrooms, age])
X_train, X_test, y_train, y_test = train_test_split(X, price, test_size=0.2)

model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Regression: House Price Prediction")
print("=" * 50)
print(f"MAE: ${mean_absolute_error(y_test, y_pred):,.0f}")
print(f"R2 Score: {r2_score(y_test, y_pred):.3f}")
print("\nSample Predictions:")
for i in range(3):
    print(f"  House: {X_test[i][0]} sqft, {X_test[i][1]} bed, {X_test[i][2]} yrs old")
    print(f"  Predicted: ${y_pred[i]:,.0f}, Actual: ${y_test[i]:,.0f}")

---

## 2. Unsupervised Learning

### 2.1 Clustering

In [ ]:
# Example: Customer Segmentation (Clustering)
from sklearn.cluster import KMeans

np.random.seed(42)

# Create 3 customer segments
seg1 = np.random.normal(loc=[500, 30], scale=[100, 5], size=(100, 2))  # High value
seg2 = np.random.normal(loc=[200, 15], scale=[50, 3], size=(100, 2))   # Medium value
seg3 = np.random.normal(loc=[50, 5], scale=[20, 2], size=(100, 2))     # Low value

data = np.vstack([seg1, seg2, seg3])
data = np.clip(data, 0, None)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(data)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
scatter = ax.scatter(data[:, 0], data[:, 1], c=clusters, cmap='viridis', alpha=0.6)
ax.set_xlabel('Average Purchase ($)')
ax.set_ylabel('Visits per Month')
ax.set_title('Customer Segmentation with K-Means')
plt.colorbar(scatter, label='Cluster')
plt.show()

print("\nCluster Sizes:", np.bincount(clusters))

### 2.2 Anomaly Detection

In [ ]:
# Example: Fraud Detection (Anomaly Detection)
from sklearn.ensemble import IsolationForest

np.random.seed(42)

# Normal transactions
normal = np.random.normal(loc=[100, 12], scale=[50, 4], size=(950, 2))
# Fraudulent transactions (anomalies)
fraud = np.random.normal(loc=[500, 2], scale=[100, 1], size=(50, 2))

X = np.vstack([normal, fraud])
y_true = np.array([0]*950 + [1]*50)

iso_forest = IsolationForest(contamination=0.05, random_state=42)
y_pred = (iso_forest.fit_predict(X) == -1).astype(int)

print("Anomaly Detection: Fraud Detection")
print("=" * 50)
print(f"Precision: {precision_score(y_true, y_pred):.2f}")
print(f"Recall: {recall_score(y_true, y_pred):.2f}")
print(f"F1 Score: {f1_score(y_true, y_pred):.2f}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['green' if p == 0 else 'red' for p in y_pred]
ax.scatter(X[:, 0], X[:, 1], c=colors, alpha=0.5)
ax.set_xlabel('Transaction Amount ($)')
ax.set_ylabel('Transaction Hour')
ax.set_title('Anomaly Detection: Normal (Green) vs Fraud (Red)')
plt.show()

---

## 3. Reinforcement Learning

In [ ]:
# Example: Multi-Armed Bandit for Content Recommendation

class MultiArmedBandit:
    def __init__(self, n_arms):
        self.n_arms = n_arms
        self.counts = np.zeros(n_arms)
        self.values = np.zeros(n_arms)
    
    def select_arm(self, epsilon=0.1):
        if np.random.random() < epsilon:
            return np.random.randint(self.n_arms)
        return np.argmax(self.values)
    
    def update(self, arm, reward):
        self.counts[arm] += 1
        self.values[arm] += (reward - self.values[arm]) / self.counts[arm]

# Simulate content types with different engagement rates
true_rates = [0.1, 0.15, 0.3, 0.25, 0.2]
content_types = ['News', 'Sports', 'Entertainment', 'Tech', 'Lifestyle']

bandit = MultiArmedBandit(n_arms=5)
rewards = []

for _ in range(1000):
    arm = bandit.select_arm(epsilon=0.1)
    reward = np.random.binomial(1, true_rates[arm])
    bandit.update(arm, reward)
    rewards.append(reward)

print("Reinforcement Learning: Content Recommendation")
print("=" * 50)
print("\nLearned vs True CTR:")
for name, true, learned in zip(content_types, true_rates, bandit.values):
    print(f"  {name}: True={true:.2f}, Learned={learned:.2f}")
print(f"\nAverage Reward: {np.mean(rewards):.3f} (Best possible: {max(true_rates):.2f})")

---

## 4. Decision Framework

In [ ]:
def select_ml_category(has_labels, output_type, feedback_type='none'):
    """Select the appropriate ML category."""
    if has_labels:
        if output_type == 'category':
            return 'Supervised - Classification'
        elif output_type == 'number':
            return 'Supervised - Regression'
        elif output_type == 'ranking':
            return 'Supervised - Ranking'
    elif feedback_type == 'delayed':
        return 'Reinforcement Learning'
    else:
        if output_type == 'groups':
            return 'Unsupervised - Clustering'
        elif output_type == 'anomalies':
            return 'Unsupervised - Anomaly Detection'
        else:
            return 'Unsupervised - Dimensionality Reduction'
    return 'Unknown'

scenarios = [
    ('Spam Detection', True, 'category', 'none'),
    ('House Price', True, 'number', 'none'),
    ('Customer Segmentation', False, 'groups', 'none'),
    ('Fraud Detection', False, 'anomalies', 'none'),
    ('Ad Optimization', False, 'actions', 'delayed'),
]

print("ML Category Selection:")
print("=" * 50)
for name, labels, output, feedback in scenarios:
    result = select_ml_category(labels, output, feedback)
    print(f"\n{name}: {result}")

In [ ]:
# Comparison table
comparison = pd.DataFrame({
    'Task Type': ['Binary Classification', 'Multi-class', 'Regression', 'Clustering', 'Anomaly Detection', 'RL'],
    'Category': ['Supervised', 'Supervised', 'Supervised', 'Unsupervised', 'Unsupervised', 'RL'],
    'Output': ['0 or 1', 'One of N', 'Continuous', 'Cluster ID', 'Normal/Anomaly', 'Actions'],
    'Key Metrics': ['AUC, F1', 'Accuracy, F1-macro', 'MSE, R2', 'Silhouette', 'Precision, Recall', 'Reward'],
    'Example': ['Spam', 'Sentiment', 'Price', 'Segmentation', 'Fraud', 'Ads']
})

print("\nTask Types Comparison:")
display(comparison)

---

## 5. Hands-On Exercise

In [ ]:
exercises = [
    ("Predict if a loan will default", "Binary Classification"),
    ("Group customers by behavior", "Clustering"),
    ("Predict stock price", "Regression"),
    ("Find network intrusions", "Anomaly Detection"),
    ("Optimize email send times", "Reinforcement Learning"),
]

print("Exercise: What ML Category?")
print("=" * 50)
for i, (scenario, answer) in enumerate(exercises, 1):
    print(f"\n{i}. {scenario}")

print("\n" + "=" * 50)
print("ANSWERS:")
for i, (_, answer) in enumerate(exercises, 1):
    print(f"{i}. {answer}")

---

## Summary

### Key Takeaways

1. **Three Main Categories**:
   - **Supervised**: Has labels, predicts for new data
   - **Unsupervised**: No labels, discovers patterns
   - **Reinforcement**: Learns from feedback/rewards

2. **Supervised Learning Tasks**:
   - Binary/Multi-class Classification
   - Regression
   - Ranking

3. **Unsupervised Learning Tasks**:
   - Clustering
   - Anomaly Detection
   - Dimensionality Reduction

4. **Decision Framework**:
   - Start with data availability (labeled or not)
   - Consider output type needed
   - Match to appropriate task

### Next Steps

You're now ready for:
- **Data Preparation**: ETL and feature engineering
- **Model Development**: Selection and training
- **Evaluation**: Metrics and testing

---

In [ ]:
# Quick Quiz
quiz = [
    ("Which ML category requires labeled data?", "A", 
     ["A) Supervised", "B) Unsupervised", "C) Reinforcement", "D) All"]),
    ("Best task for predicting prices?", "C",
     ["A) Classification", "B) Clustering", "C) Regression", "D) Anomaly"]),
    ("When to use RL?", "D",
     ["A) Lots of labels", "B) Need clusters", "C) Continuous output", "D) Delayed feedback"])
]

print("Quick Quiz")
print("=" * 40)
for i, (q, _, opts) in enumerate(quiz, 1):
    print(f"\nQ{i}: {q}")
    for opt in opts:
        print(f"   {opt}")

print("\n" + "=" * 40)
print("Answers: 1-A, 2-C, 3-D")